# SQP method

<i>Group 08<br>
Participants information in alphabetical order</i>
<table style="width:100%">
  <
    <th style = "text-align: left">#</th>
    <th style = "text-align: left">Name</th>
    <th style = "text-align: left">Lastname</th>
    <th style = "text-align: left">Matr Number</th>
  </tr>
  <tr>
    <td style = "text-align: left">1</td>
    <td style = "text-align: left">Alexander</td>
    <td style = "text-align: left">Temper</td>
    <td style = "text-align: left">K11905007</td>
  </tr>
  <tr>
    <td style = "text-align: left">2</td>
    <td style = "text-align: left">Bernhard Michael</td>
    <td style = "text-align: left">Voggenberger</td>
    <td style = "text-align: left">K11907093</td>
  </tr>
  <tr>
    <td style = "text-align: left">3</td>
    <td style = "text-align: left">Christian</td>
    <td style = "text-align: left">Ganhör</td>
    <td style = "text-align: left">K11911652</td>
  </tr>
  <tr>
    <td style = "text-align: left">4</td>
    <td style = "text-align: left">Christoph</td>
    <td style = "text-align: left">Koller</td>
    <td style = "text-align: left">K11910272</td>
    </tr>
  <tr>
    <td style = "text-align: left">5</td>
    <td style = "text-align: left">Franziska</td>
    <td style = "text-align: left">Denk</td>
    <td style = "text-align: left">K11904292</td>
  </tr>
  <tr>
    <td style = "text-align: left">6</td>
    <td style = "text-align: left">Lukas</td>
    <td style = "text-align: left">Gattermayr</td>
    <td style = "text-align: left">K11911639</td>
  </tr>
  <tr>
    <td style = "text-align: left">7</td>
    <td style = "text-align: left">Nathanael</td>
    <td style = "text-align: left">Harmetzky</td>
    <td style = "text-align: left">K11916566</td>
  </tr>
  <tr>
    <td style = "text-align: left">8</td>
    <td style = "text-align: left">Raphael-Pascal</td>
    <td style = "text-align: left">Endstrasser</td>
    <td style = "text-align: left">K11907909</td>
  </tr>
  <tr>
    <td style = "text-align: left">9</td>
    <td style = "text-align: left">Tobias</td>
    <td style = "text-align: left">Stierberger</td>
    <td style = "text-align: left">K11907869</td>
  </tr>
  <tr>
    <td style = "text-align: left">10</td>
    <td style = "text-align: left">***</td>
    <td style = "text-align: left">***</td>
    <td style = "text-align: left">***</td>
  </tr>
</table>


In [1]:
import numpy as np
import scipy 
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from typing import Callable, Tuple
from dataclasses import dataclass

In [31]:
# As we are using additional .py files, enable their reloading without restarting the kernel
%load_ext autoreload
%autoreload 2

from shared.classes import MinimizationProblem, Constraint
from sqp.problems import create_example_18_3_problem

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Implementation

In [32]:
problem = create_example_18_3_problem()

In [28]:
# prototyping algorithm 18.3

# TODO implement quadratic minimization
def solve_quadratic(problem):
    p = (1,1,1,1,1)
    l = (0,0,0)
    return p, l

# should be done
def find_mu(prev_mu, f_grad, p, sigma, L_hessian, c_norm, rho=0.5):
    hessian_quadratic_p = p @ L_hessian @ p
    sigma = 1 if hessian_quadratic_p > 0 else 0

    inequality_18_36 = (f_grad @ p + (sigma/2) * hessian_quadratic_p) / ((1-rho) * c_norm)
    if prev_mu is not None and prev_mu >= inequality_18_36:
        return prev_mu
    
    return inequality_18_36 + 1

# should be done (phi_1 function in book)
def l1_merit(f_x, mu, c_norm):
    return f_x + mu * c_norm

# should be done (D_1(phi_1) function in book)
def l1_merit_directional_gradient(f_grad, p, mu, c_norm):
    return f_grad @ p - mu * c_norm

# should be done
def transform_sqp_to_quadr_constraint(constraint, c_i, c_i_grad):
    def transformed_constraint(p):
        return c_i_grad @ p + c_i

    return Constraint(transformed_constraint, equality=constraint.equality)

# should be done except x0
def create_iterate_quadratic_problem(problem, f_x, f_grad, c, A, L_hessian):
    def f(p):
        return f_x + f_grad @ p + 1/2 * p @ L_hessian @ p

    constraints = np.array([transform_sqp_to_quadr_constraint(constraint, c_i, c_i_grad) for constraint, c_i, c_i_grad in zip(problem.constraints, c, A)])
    # TODO how to calc x0???

    return MinimizationProblem(f=f, constraints=constraints, solution=None, x0=None)

# should be done if meant that way?
def kkt_fulfilled(problem: MinimizationProblem, x, l, c, tolerance=1e-5):
    # 12.34d
    if np.any(l + tolerance < 0):
        return False
    
    for l_i, constraint, c_i in zip(l, problem.constraints, c):
        # 12.34b
        if constraint.equality:
            if np.any(abs(c_i) > tolerance):
                return False
        # 12.34c
        else:
            if np.any(c_i + tolerance < 0):
                return False

        #12.34e
        if np.any(abs(l_i * c_i ) > tolerance):
            return False
        
    l_gradient = problem.calc_lagrangian_gradient_at(x, l)

    # 12.34a
    if np.any(abs(l_gradient) > tolerance):
        return False

    return True

# should be done
def backtracking_line_search_sqp(problem, eta=0.3, tau=0.5, tolerance = 1e-5)
    x = problem.x0
    l = (0, 0, 0)

    f_x = problem.calc_f_at(x)
    f_grad = problem.calc_gradient_at(x)
    c = problem.calc_constraints_at(x)
    c_norm = np.linalg.norm(c, ord=1)
    A = problem.calc_constraints_jacobian_at(x)

    # TODO quasi-newton approx 
    L_hessian = problem.calc_lagrangian_hessian_at(x, l)

    while True:
        quadr_problem = create_iterate_quadratic_problem(problem, f_x, f_grad, c, A, L_hessian)
        p, l_hat = solve_quadratic(quadr_problem)
        p_lambda = l_hat - l
        mu = find_mu()

        merit_at_x = l1_merit(x, mu, c_norm)
        dir_gradient_merit = l1_merit_directional_gradient(f_grad, p, mu, c_norm)
        first_wolfe_condition_term = merit_at_x + eta * alpha * dir_gradient_merit

        alpha = 1
        while l1_merit(x + alpha * p, mu, c_norm) > first_wolfe_condition_term:
            alpha *= tau
        
        x += alpha * p
        l += alpha * p_lambda

        f_x = problem.calc_f_at(x)
        f_grad = problem.calc_gradient_at(x)
        c = problem.calc_constraints_at(x)
        c_norm = np.linalg.norm(c, ord=1)
        A = problem.calc_constraints_jacobian_at(x)

        # TODO quasi-newton approx 
        L_hessian = problem.calc_lagrangian_hessian_at(x, l)

        if kkt_fulfilled(problem, x, l, c, tolerance)
            break

    return x, l


array([[-3.42000002,  3.18000001,  3.63999999, -1.52600002, -1.52600002],
       [ 0.        ,  1.81999999,  1.59      ,  3.815     ,  3.815     ],
       [ 8.7723    ,  7.58429998,  0.        ,  0.        ,  0.        ]])